## Fourth Movement: Random Defender with CVE Data

In [74]:
# Import required libraries
import networkx as nx
import numpy as np
from scipy.special import factorial
from datetime import datetime
import logging
from scipy.stats import geom
import sys
import os

In [75]:
import os
experiment_log_file = 'experiment_4.log'
log_path = os.path.join(os.getcwd(), experiment_log_file)

if os.path.exists(log_path):
    os.remove(log_path)

logger = logging.getLogger()
handler = logging.FileHandler(log_path, mode='w')
handler.setFormatter(logging.Formatter('%(message)s'))
logger.addHandler(handler)
logger.setLevel(logging.INFO)

logger.info(f'[1] {experiment_log_file}')
logger.info(f'[1] "{datetime.now().strftime("%a %b %d %H:%M:%S %Y")}"')

In [76]:
# Global configuration
DEFAULT_WEIGHT_VALUE = 0  # Can be changed to 1 if needed

In [77]:
%run attack_graph_MIR100.ipynb

### Combined Random-Interval and Exploit-Dependent Distribution Explanations
#### Base Formula
This distribution models the probability of completing n exploits within a random time window, where both the defender's check interval and exploit difficulties vary:
\begin{equation}
f_N(n) = 
\begin{cases}
\left(\frac{\lambda_\pi}{\lambda_\pi + \lambda_D}\right)^n & \text{if } n \geq 1 \\
1 - Pr(E_1 \leq W) & \text{if } n = 0
\end{cases}
\end{equation}
Where $\lambda_\pi = 1/\bar{q}$ is the attack rate derived from exploit difficulties, and $\lambda_D$ is the defender's check rate.

#### Definition of Parameters in our Specific Case
In the code:
- Exploit success probabilities are derived from edge weights
- A geometric mean of these probabilities ($\bar{q}$) is used to calculate attack_rate
- The probability parameter is calculated as $p = \frac{\text{attack\_rate}}{\text{attack\_rate} + \text{defense\_rate}}$

This corresponds to formula (13) in the paper, which simplifies to another geometric distribution with modified parameters.

#### Basic Explanation
This distribution combines the unpredictable defender checking pattern with varying exploit difficulties:

- Defender checks at random times (exponentially distributed intervals)
- Each exploit has different difficulty (derived from CVE data)
- attack_rate is calculated as inverse of geometric mean of exploit probabilities
- defense_rate directly impacts how far attacker can progress
- Higher defense_rate → attacker progress more limited
- Lower defense_rate → attacker can attempt more difficult exploits

Key insight: This model captures the most realistic scenario where defenders have random checking patterns and attackers face varying technical challenges.

This fits our intuition: In real-world environments, security teams respond at unpredictable intervals while attackers encounter exploits of varying difficulty, creating a complex interplay that this distribution effectively models.

### Run the Code

In [78]:
attack_rate_list = [None]   
# defense_rate_list = [1,2,3]  
defense_rate_list = [1]  

In [79]:
def random_steps(route, attack_rate=None, defense_rate=None, graph=None):
    """
    Calculates probabilities for attacker movement along route.
    Returns probability distribution over possible ending nodes.
    """
    # Calculate hardness values for each edge 
    hardness = []
    for i in range(len(route) - 1):
        start_node = route[i]
        end_node = route[i + 1]
        
        # Initialize variables for max weight loop
        weights = []
        # Collect all weights for max
        for edge in graph[start_node][end_node].values():
            weights.append(edge.get('weight', DEFAULT_WEIGHT_VALUE))
        # Get maximum weight
        max_weight = max(weights) if weights else DEFAULT_WEIGHT_VALUE
        
        # Initialize variables for min weight loop
        min_weights = []
        # Collect all weights for min
        for edge in graph[start_node][end_node].values():
            min_weights.append(edge.get('weight', DEFAULT_WEIGHT_VALUE))
        # Get minimum weight
        min_weight = min(min_weights) if min_weights else DEFAULT_WEIGHT_VALUE
            
        # Convert weights to probabilities
        # We could take max_weight or min_weight here
        # hardness.append(np.exp(-max_weight))

        # Important: We use min_weight here because of the following reason:
        # Since the formula to calculate hardness in R is hardness = exp(-weight)
        # taking the minimum weight will give us the maximum hardness
        # which translates to the path being EASIEST to traverse.
        # Yes hardness of 1 means path is trivial, hardness 0 means path is impossible
        hardness.append(np.exp(-min_weight))

    
    # Convert to arrays
    hardness = np.array(hardness)

    # print(f'Hardness: {hardness}')
    
    # Calculate attack rate using geometric mean if not provided
    if attack_rate is None:
        # Geometric mean function (mirrors the R geomean function)
        def geom_mean(x):
            # Filter out zeros to avoid log(0)
            positive_values = x[x > 0]
            if len(positive_values) == 0:
                return 1  # Default if all values are zero or array is empty
            return np.exp(np.mean(np.log(positive_values)))
        
        attack_rate = 1 / geom_mean(hardness)
    
    # Calculate probability distribution using geometric distribution
    # Equivalent to R's dgeom function
    prob = attack_rate / (attack_rate + defense_rate)
    
    # Create range of values from 0 to length(route)-1
    x_values = np.arange(len(route))
    
    # Calculate probability mass function (pmf) for geometric distribution
    # pdf_d = geom.pmf(x_values, prob)
    pdf_d = prob * (1-prob)**x_values
    
    # Normalize to ensure probabilities sum to 1
    pdf_d = pdf_d / np.sum(pdf_d)
    
    # print(f"This is the final pdf that is returned in the end: {pdf_d}")
    return pdf_d

# Example usage:
# route = [1, 2, 3, 4]
# defense_rate = 2
# pdf = random_steps(route, attack_graph=G, defense_rate=defense_rate)

In [80]:
# %run ctr-core_simple.ipynb
%run ctr-core_tests.ipynb
main()


After merging targets:
Nodes: [1, 5, 15, 11, 3, 6, 8, 4, 7, 2, 9, 10, 0, 'c(12,13,14,16)']
Edges with weights:
1 -> 5 (key=0) : 2.1958405355640576
5 -> 15 (key=0) : 2.1958405355640576
15 -> c(12,13,14,16) (key=0) : 0.7489220813074156
15 -> c(12,13,14,16) (key=1) : 0.7489220813074156
15 -> c(12,13,14,16) (key=2) : 0.7489220813074156
11 -> c(12,13,14,16) (key=0) : 1.064439873679208
11 -> c(12,13,14,16) (key=1) : 0.7489220813074156
11 -> c(12,13,14,16) (key=2) : 0.0
3 -> 6 (key=0) : 1.064439873679208
3 -> 8 (key=0) : 0.7489220813074156
6 -> 8 (key=0) : 0.0
8 -> c(12,13,14,16) (key=0) : 0.0
8 -> c(12,13,14,16) (key=1) : 0.7489220813074156
8 -> 10 (key=0) : 0.0
4 -> 7 (key=0) : 0.7489220813074156
7 -> c(12,13,14,16) (key=0) : 0.7489220813074156
7 -> c(12,13,14,16) (key=1) : 0.7489220813074156
7 -> 10 (key=0) : 0.7489220813074156
2 -> c(12,13,14,16) (key=0) : 1.064439873679208
2 -> 9 (key=0) : 0.7489220813074156
2 -> 10 (key=0) : 1.064439873679208
2 -> 11 (key=0) : 0.7489220813074156
9 -> c

In [81]:
with open(experiment_log_file, 'r') as f:
    print(f.read())

[1] experiment_4.log
[1] "Thu Feb 27 19:14:38 2025"

++++++++++++++++++++++++++++++++

The virtual target nodeID is c(12,13,14,16)

attack rate =  None , defense rate =  1 

	equilibrium for multiobjective security game (MOSG)

optimal defense strategy:
         prob.
10 0.000000e+00
11 3.117287e-01
15 2.283505e-01
5 0.000000e+00
6 0.000000e+00
7 4.236365e-02
8 4.175572e-01
9 0.000000e+00

worst case attack strategies per goal:
          1
1 0.0000000
2 0.2019000
3 0.0000000
4 0.0000000
5 0.0000000
6 0.2937196
7 0.2657957
8 0.0000000
9 0.0000000
10 0.0000000
11 0.2385847
[1] 0.025

Defender can keep attacker success below: 0.025
Attacker can guarantee success probability of: 0.025

